# 02. アニメーション可視化

時間変化する音響データのアニメーション表示を学びます。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy import signal
from scipy.fft import fft, fftfreq
import warnings
warnings.filterwarnings('ignore')

## 1. リアルタイムスペクトラムアナライザー

In [ ]:
class RealtimeSpectrumAnalyzer:
    def __init__(self, sample_rate=8000, window_size=1024):
        self.sample_rate = sample_rate
        self.window_size = window_size
        self.frequencies = fftfreq(window_size, 1/sample_rate)[:window_size//2]
        
        # アニメーション用データ
        self.time_index = 0
        self.spectrum_history = []
        
        # テスト信号生成器
        self.signal_generator = self.create_test_signal()
        
    def create_test_signal(self):
        """アニメーション用のテスト信号生成器"""
        duration = 10.0
        total_samples = int(self.sample_rate * duration)
        t = np.linspace(0, duration, total_samples)
        
        # 複数の周波数成分が時間変化する信号
        signal_data = np.zeros(total_samples)
        
        # 基本周波数の変化
        f1 = 200 + 100 * np.sin(2 * np.pi * 0.5 * t)  # 200-300Hz で変化
        f2 = 400 + 50 * np.sin(2 * np.pi * 0.3 * t + np.pi/4)  # 350-450Hz で変化
        f3 = 600 + 200 * np.sin(2 * np.pi * 0.2 * t + np.pi/2)  # 400-800Hz で変化
        
        # 時間変化する振幅
        a1 = 0.5 * (1 + np.sin(2 * np.pi * 0.7 * t))
        a2 = 0.3 * (1 + np.sin(2 * np.pi * 0.4 * t + np.pi))
        a3 = 0.2 * (1 + np.sin(2 * np.pi * 0.6 * t + np.pi/3))
        
        # 信号合成
        for i in range(total_samples):
            signal_data[i] = (a1[i] * np.sin(2 * np.pi * f1[i] * t[i]) +
                             a2[i] * np.sin(2 * np.pi * f2[i] * t[i]) +
                             a3[i] * np.sin(2 * np.pi * f3[i] * t[i]))
        
        # ノイズ追加
        np.random.seed(42)
        noise = 0.1 * np.random.normal(0, 1, total_samples)
        signal_data += noise
        
        return signal_data
        
    def get_next_frame(self):
        """次のフレームのデータを取得"""
        start_idx = self.time_index * (self.window_size // 4)
        end_idx = start_idx + self.window_size
        
        if end_idx > len(self.signal_generator):
            self.time_index = 0  # ループ
            start_idx = 0
            end_idx = self.window_size
        
        frame = self.signal_generator[start_idx:end_idx]
        
        # 窓関数適用
        windowed_frame = frame * np.hanning(len(frame))
        
        # FFT
        fft_result = fft(windowed_frame)
        magnitude = np.abs(fft_result[:self.window_size//2])
        
        self.time_index += 1
        
        return frame, magnitude

# アニメーション設定
analyzer = RealtimeSpectrumAnalyzer()

# アニメーション表示のセットアップ
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# 波形プロット
time_axis = np.arange(analyzer.window_size) / analyzer.sample_rate
waveform_line, = ax1.plot(time_axis, np.zeros(analyzer.window_size), 'b-', linewidth=2)
ax1.set_xlim(0, max(time_axis))
ax1.set_ylim(-2, 2)
ax1.set_title('リアルタイム波形')
ax1.set_xlabel('時間 (秒)')
ax1.set_ylabel('振幅')
ax1.grid(True)

# スペクトラムプロット
spectrum_line, = ax2.plot(analyzer.frequencies, np.zeros(len(analyzer.frequencies)), 'r-', linewidth=2)
ax2.set_xlim(0, 1000)
ax2.set_ylim(0, 100)
ax2.set_title('リアルタイムスペクトラム')
ax2.set_xlabel('周波数 (Hz)')
ax2.set_ylabel('振幅')
ax2.grid(True)

def animate(frame_num):
    """アニメーション更新関数"""
    waveform, spectrum = analyzer.get_next_frame()
    
    # 波形更新
    waveform_line.set_ydata(waveform)
    
    # スペクトラム更新
    spectrum_line.set_ydata(spectrum)
    
    # Y軸の自動調整
    ax2.set_ylim(0, max(spectrum) * 1.1 if max(spectrum) > 0 else 1)
    
    return waveform_line, spectrum_line

# アニメーション作成（表示用に最初の数フレームを静止画で示す）
# 実際の実行時はコメントアウトを外してください
# anim = animation.FuncAnimation(fig, animate, frames=200, interval=50, blit=True, repeat=True)
# plt.show()

# 代替として、複数フレームの静止画を表示
fig_frames, axes = plt.subplots(2, 3, figsize=(18, 10))
frames_to_show = [0, 20, 40, 60, 80, 100]

for i, frame_num in enumerate(frames_to_show):
    analyzer.time_index = frame_num
    waveform, spectrum = analyzer.get_next_frame()
    
    row = i // 3
    col = i % 3
    
    # 波形
    if row == 0:
        axes[row, col].plot(time_axis, waveform, 'b-', linewidth=2)
        axes[row, col].set_title(f'波形 (フレーム {frame_num})')
        axes[row, col].set_ylabel('振幅')
        axes[row, col].grid(True)
        axes[row, col].set_ylim(-2, 2)
    
    # スペクトラム
    else:
        axes[row, col].plot(analyzer.frequencies, spectrum, 'r-', linewidth=2)
        axes[row, col].set_title(f'スペクトラム (フレーム {frame_num})')
        axes[row, col].set_xlabel('周波数 (Hz)')
        axes[row, col].set_ylabel('振幅')
        axes[row, col].grid(True)
        axes[row, col].set_xlim(0, 1000)

plt.tight_layout()
plt.show()

print("リアルタイムアニメーションの特徴:")
print("- 動的な波形とスペクトラム表示")
print("- 周波数成分の時間変化を観察")
print("- インタラクティブな更新")
print("- 複数の表示モードの同期")

## 2. 3Dスペクトログラムアニメーション

In [ ]:
# 3Dスペクトログラムの時間発展アニメーション
class SpectrogramAnimator:
    def __init__(self, sample_rate=4000):
        self.sample_rate = sample_rate
        self.create_test_signal()
        
    def create_test_signal(self):
        """チャープ信号とマルチトーンの組み合わせ"""
        duration = 6.0
        t = np.linspace(0, duration, int(self.sample_rate * duration))
        
        # セクション1: 上昇チャープ
        section1 = signal.chirp(t[t < 2], 100, 2, 800, method='linear')
        
        # セクション2: 複数トーン
        t2 = t[(t >= 2) & (t < 4)] - 2
        section2 = (np.sin(2 * np.pi * 200 * t2) + 
                   np.sin(2 * np.pi * 400 * t2) + 
                   np.sin(2 * np.pi * 600 * t2))
        
        # セクション3: 下降チャープ
        t3 = t[t >= 4] - 4
        section3 = signal.chirp(t3, 800, 2, 150, method='linear')
        
        self.signal = np.concatenate([section1, section2, section3])
        self.time = t
        
        # スペクトログラム計算
        self.f, self.t_spec, self.Sxx = signal.spectrogram(
            self.signal, self.sample_rate, nperseg=256, noverlap=128
        )
        
    def create_animation_frames(self, num_frames=20):
        """アニメーション用フレーム作成"""
        frames = []
        
        for i in range(num_frames):
            # 時間の進行
            time_progress = i / (num_frames - 1)
            current_time_idx = int(time_progress * (len(self.t_spec) - 1))
            
            # 現在時刻までのスペクトログラムデータ
            current_spec = self.Sxx[:, :current_time_idx + 1]
            current_times = self.t_spec[:current_time_idx + 1]
            
            frames.append({
                'times': current_times,
                'freqs': self.f,
                'spectrogram': current_spec,
                'current_time': self.t_spec[current_time_idx],
                'progress': time_progress
            })
            
        return frames

# アニメーター作成
animator = SpectrogramAnimator()
frames = animator.create_animation_frames(12)

# フレーム表示（アニメーションの代替）
fig, axes = plt.subplots(3, 4, figsize=(20, 15))

for i, frame in enumerate(frames):
    row = i // 4
    col = i % 4
    
    if i < 12:  # 12フレームまで表示
        ax = axes[row, col]
        
        # スペクトログラム表示
        Sxx_db = 10 * np.log10(frame['spectrogram'] + 1e-10)
        
        im = ax.pcolormesh(frame['times'], frame['freqs'], Sxx_db,
                          shading='gouraud', cmap='viridis')
        
        # 現在時刻をマーク
        ax.axvline(x=frame['current_time'], color='red', linewidth=3, alpha=0.8)
        
        ax.set_title(f'フレーム {i+1} (t={frame["current_time"]:.1f}s)')
        ax.set_xlabel('時間 (秒)')
        ax.set_ylabel('周波数 (Hz)')
        ax.set_ylim(0, 1000)
        
        # 進捗バー
        progress_text = f"進捗: {frame['progress']*100:.0f}%"
        ax.text(0.02, 0.95, progress_text, transform=ax.transAxes,
               bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8),
               fontsize=10)

plt.tight_layout()
plt.show()

# 元の信号も表示
plt.figure(figsize=(15, 6))

plt.subplot(2, 1, 1)
plt.plot(animator.time, animator.signal, 'b-', linewidth=1)
plt.title('元の信号（全体）')
plt.xlabel('時間 (秒)')
plt.ylabel('振幅')
plt.grid(True)

plt.subplot(2, 1, 2)
plt.pcolormesh(animator.t_spec, animator.f, 
              10 * np.log10(animator.Sxx + 1e-10),
              shading='gouraud', cmap='viridis')
plt.title('完全なスペクトログラム')
plt.xlabel('時間 (秒)')
plt.ylabel('周波数 (Hz)')
plt.ylim(0, 1000)
plt.colorbar(label='振幅 (dB)')

plt.tight_layout()
plt.show()

print("スペクトログラムアニメーションの効果:")
print("- 時間発展の視覚的理解")
print("- 音響イベントの時系列追跡")
print("- 周波数変化の動的観察")
print("- 教育・プレゼンテーション効果")

## 3. 音響パラメータの動的変化

In [ ]:
# 音響パラメータの時間変化アニメーション
class ParameterAnimator:
    def __init__(self, sample_rate=8000):
        self.sample_rate = sample_rate
        self.create_analysis_data()
        
    def create_analysis_data(self):
        """分析用データの生成"""
        duration = 5.0
        t = np.linspace(0, duration, int(self.sample_rate * duration))
        
        # 複雑な信号の生成
        # 基本周波数の変化
        f0 = 220 * (1 + 0.3 * np.sin(2 * np.pi * 0.5 * t))
        
        # 振幅エンベロープ
        envelope = 0.5 * (1 + np.sin(2 * np.pi * 0.3 * t + np.pi/4))
        
        # 高調波成分
        signal = np.zeros_like(t)
        for harmonic in range(1, 6):
            amplitude = envelope / harmonic
            frequency = f0 * harmonic
            signal += amplitude * np.sin(2 * np.pi * frequency * t)
        
        # フレームごとの分析
        frame_size = 1024
        hop_size = frame_size // 4
        
        self.times = []
        self.centroids = []
        self.bandwidths = []
        self.energies = []
        self.zcr_values = []
        self.pitches = []
        
        for i in range(0, len(signal) - frame_size, hop_size):
            frame = signal[i:i + frame_size]
            time = i / self.sample_rate
            self.times.append(time)
            
            # スペクトル重心
            windowed = frame * np.hanning(frame_size)
            spectrum = np.abs(fft(windowed))
            freqs = fftfreq(frame_size, 1/self.sample_rate)
            positive_freqs = freqs[:frame_size//2]
            positive_spectrum = spectrum[:frame_size//2]
            
            if np.sum(positive_spectrum) > 0:
                centroid = np.sum(positive_freqs * positive_spectrum) / np.sum(positive_spectrum)
                bandwidth = np.sqrt(np.sum(((positive_freqs - centroid) ** 2) * positive_spectrum) / 
                                  np.sum(positive_spectrum))
            else:
                centroid = 0
                bandwidth = 0
                
            self.centroids.append(centroid)
            self.bandwidths.append(bandwidth)
            
            # エネルギー
            energy = np.sum(frame ** 2) / frame_size
            self.energies.append(energy)
            
            # ゼロクロッシング率
            zcr = np.sum(np.diff(np.sign(frame)) != 0) / (frame_size - 1)
            self.zcr_values.append(zcr)
            
            # ピッチ（簡易推定）
            autocorr = np.correlate(frame, frame, mode='full')
            autocorr = autocorr[len(autocorr)//2:]
            if autocorr[0] > 0:
                autocorr = autocorr / autocorr[0]
            
            min_period = int(self.sample_rate / 500)
            max_period = int(self.sample_rate / 80)
            
            if max_period < len(autocorr):
                search_range = autocorr[min_period:max_period]
                if len(search_range) > 0:
                    max_idx = np.argmax(search_range)
                    period = max_idx + min_period
                    pitch = self.sample_rate / period
                else:
                    pitch = 0
            else:
                pitch = 0
                
            self.pitches.append(pitch)
        
        self.signal = signal
        self.time_signal = t
        
    def create_parameter_frames(self, num_frames=15):
        """パラメータアニメーション用フレーム"""
        frames = []
        
        for i in range(num_frames):
            progress = i / (num_frames - 1)
            current_idx = int(progress * (len(self.times) - 1))
            
            frames.append({
                'progress': progress,
                'current_time': self.times[current_idx],
                'times': self.times[:current_idx + 1],
                'centroids': self.centroids[:current_idx + 1],
                'bandwidths': self.bandwidths[:current_idx + 1],
                'energies': self.energies[:current_idx + 1],
                'zcr_values': self.zcr_values[:current_idx + 1],
                'pitches': self.pitches[:current_idx + 1]
            })
            
        return frames

# パラメータアニメーター
param_animator = ParameterAnimator()
param_frames = param_animator.create_parameter_frames()

# フレーム表示
fig, axes = plt.subplots(3, 5, figsize=(25, 15))

parameters = ['centroids', 'bandwidths', 'energies', 'zcr_values', 'pitches']
param_labels = ['スペクトル重心 (Hz)', '帯域幅 (Hz)', 'エネルギー', 'ZCR', 'ピッチ (Hz)']
colors = ['red', 'green', 'blue', 'orange', 'purple']

frame_indices = [0, 4, 8, 12, 14]  # 表示するフレーム

for row, frame_idx in enumerate(frame_indices):
    frame = param_frames[frame_idx]
    
    for col, (param, label, color) in enumerate(zip(parameters, param_labels, colors)):
        ax = axes[row, col]
        
        # 全データを薄く表示
        ax.plot(param_animator.times, getattr(param_animator, param), 
               color='lightgray', linewidth=1, alpha=0.5)
        
        # 現在までのデータを強調表示
        if len(frame['times']) > 0:
            ax.plot(frame['times'], frame[param], color=color, linewidth=3)
            
            # 現在地点をマーク
            if len(frame[param]) > 0:
                ax.plot(frame['current_time'], frame[param][-1], 
                       'o', color=color, markersize=10, markeredgecolor='black', markeredgewidth=2)
        
        # 現在時刻の縦線
        ax.axvline(x=frame['current_time'], color='red', linestyle='--', alpha=0.7)
        
        ax.set_title(f'{label}\n(t={frame["current_time"]:.1f}s)')
        ax.set_xlabel('時間 (秒)')
        ax.set_ylabel(label)
        ax.grid(True, alpha=0.3)
        
        # Y軸の範囲設定
        if param == 'pitches':
            ax.set_ylim(100, 400)
        elif param == 'centroids':
            ax.set_ylim(0, 1000)
        elif param == 'bandwidths':
            ax.set_ylim(0, 500)
        elif param == 'energies':
            ax.set_ylim(0, max(param_animator.energies) * 1.1)
        elif param == 'zcr_values':
            ax.set_ylim(0, max(param_animator.zcr_values) * 1.1)

plt.tight_layout()
plt.show()

# 元の信号も表示
plt.figure(figsize=(15, 6))
plt.plot(param_animator.time_signal, param_animator.signal, 'b-', linewidth=1)
plt.title('元の音響信号')
plt.xlabel('時間 (秒)')
plt.ylabel('振幅')
plt.grid(True)
plt.show()

print("パラメータアニメーションの利点:")
print("- 複数特徴量の同時観察")
print("- 時間発展の直感的理解")
print("- パターン認識の支援")
print("- 相関関係の可視化")
print("- 動的な閾値設定の支援")

## 練習問題

1. 音楽のビートに同期したビジュアライザーアニメーションを作成してみましょう
2. 3D空間での音源移動をアニメーションで表現してみましょう
3. リアルタイム音声認識の過程をアニメーションで可視化してみましょう